In [ ]:
from benchmark import Benchmark

bm = Benchmark('Fraunhofer Propagation', 'N', [2**i for i in range(6, 13)], t_min=0.5)

In [ ]:
# Physical parameters
F_number = 10
wavelength = 632e-9 # m
diameter = 10e-3 # m

focal_length = F_number * diameter
resolution_element = wavelength * F_number

# Numerical parameters
N = 1024
q = 1
num_airy =  32

In [ ]:
import hcipy

def fraunhofer_hcipy(it, N):
    pupil_grid = hcipy.make_pupil_grid(N, diameter=diameter)

    aperture = hcipy.make_circular_aperture(diameter=diameter)(pupil_grid)
    wf_hcipy = hcipy.Wavefront(aperture, wavelength)

    focal_grid = hcipy.make_focal_grid(q, num_airy, resolution_element)
    prop = hcipy.FraunhoferPropagator(pupil_grid, focal_grid, focal_length)

    for _ in it:
        wf_out = prop(wf_hcipy)

    psf = wf_out.intensity.shaped
    psf /= psf.max()

    return psf

bm.run(fraunhofer_hcipy, 'hcipy')

In [ ]:
from prysm.coordinates import make_xy_grid, cart_to_polar
from prysm.geometry import circle
from prysm.propagation import Wavefront

def fraunhofer_prysm(it, N):
    xi, eta = make_xy_grid(N, diameter=diameter * 1e3)

    r, t = cart_to_polar(xi, eta)
    A = circle(diameter * 1e3 / 2, r)

    dx = xi[0, 1] - xi[0, 0]

    wf_prysm = Wavefront.from_amp_and_phase(A, None, wavelength * 1e6, dx)

    for _ in it:
        E = wf_prysm.focus_fixed_sampling(focal_length * 1e3, resolution_element / q * 1e6, 2 * num_airy * q)

    psf = E.intensity.data
    psf /= psf.max()

    return psf

bm.run(fraunhofer_prysm, 'prysm')

In [ ]:
import poppy
import numpy as np

def fraunhofer_poppy(it, N):
    arcsec_per_px = np.rad2deg(wavelength / diameter) * 3600 / q
    fov_arcsec = num_airy * arcsec_per_px * q * 2

    wf = poppy.Wavefront(wavelength, npix=N, oversample=1)
    wf *= poppy.CircularAperture(radius=diameter / 2)

    detector = poppy.Detector(arcsec_per_px, fov_arcsec=fov_arcsec, oversample=1)

    for _ in it:
        wf2 = wf.copy()
        wf2.propagate_to(detector)

    psf = wf2.intensity
    psf /= psf.max()

    return psf

bm.run(fraunhofer_poppy, 'poppy')

In [ ]:
bm.plot()